In [1]:
import tkinter as tk
from tkinter import filedialog, Label
from ultralytics import YOLO
import cv2
from PIL import Image, ImageTk
import numpy as np
from threading import Thread

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Use your desired YOLO model weights

# Create main window
window = tk.Tk()
window.title("YOLOv8 Object Detection")
window.geometry("500x500")
window.configure(bg="#2C2F33")

# Global variables for video source
video_source = None
running = False

# Functions
def select_video():
    global video_source
    video_source = filedialog.askopenfilename(
        title="Select Video File",
        filetypes=[("Video Files", "*.mp4 *.avi *.mkv")]
    )
    if video_source:
        status_label.config(text=f"Selected Video: {video_source}", fg="white")
    else:
        status_label.config(text="No video selected.", fg="red")

def start_detection():
    global running
    running = True
    if video_source or video_source == 0:  # Check if a video source is available
        Thread(target=run_detection).start()
    else:
        status_label.config(text="Please select a video source!", fg="red")

def stop_detection():
    global running
    running = False
    status_label.config(text="Detection stopped.", fg="yellow")

def run_detection():
    global running
    cap = cv2.VideoCapture(video_source if video_source != 0 else 0)

    if not cap.isOpened():
        status_label.config(text="Error: Could not open video source.", fg="red")
        return

    while running:
        ret, frame = cap.read()
        if not ret:
            status_label.config(text="Video ended or no frame captured.", fg="yellow")
            break

        # Perform detection
        results = model(frame)
        annotated_frame = results[0].plot()

        # Display frame
        cv2.imshow("YOLOv8 Detection", annotated_frame)

        # Stop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def set_webcam():
    global video_source
    video_source = 0
    status_label.config(text="Using Webcam.", fg="white")

def select_image():
    file_path = filedialog.askopenfilename(
        title="Select Image File",
        filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
    )
    if file_path:
        detect_from_image(file_path)
    else:
        status_label.config(text="No image selected.", fg="red")

def detect_from_image(image_path):
    img = cv2.imread(image_path)
    results = model(img)
    annotated_img = results[0].plot()

    # Convert annotated image to a format that can be displayed in Tkinter
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    annotated_img_pil = Image.fromarray(annotated_img)
    annotated_img_tk = ImageTk.PhotoImage(annotated_img_pil)

    # Create a new window to display the annotated image
    img_window = tk.Toplevel(window)
    img_window.title("Detection Results")
    img_window.configure(bg="#2C2F33")

    img_label = tk.Label(img_window, image=annotated_img_tk, bg="#2C2F33")
    img_label.image = annotated_img_tk  # Keep a reference to avoid garbage collection
    img_label.pack()

# Widgets with dark theme
font_title = ("Helvetica", 18, "bold")
font_buttons = ("Helvetica", 12)
font_status = ("Helvetica", 10)

# Title
title_label = Label(window, text="YOLOv8 Object Detection", font=font_title, bg="#23272A", fg="white")
title_label.pack(pady=20)

# Buttons
select_button = tk.Button(window, text="Select Video", command=select_video, font=font_buttons, bg="#7289DA", fg="white", relief="flat")
select_button.pack(pady=10)

webcam_button = tk.Button(window, text="Use Webcam", command=set_webcam, font=font_buttons, bg="#7289DA", fg="white", relief="flat")
webcam_button.pack(pady=10)

image_button = tk.Button(window, text="Select Image", command=select_image, font=font_buttons, bg="#7289DA", fg="white", relief="flat")
image_button.pack(pady=10)

start_button = tk.Button(window, text="Start Detection", command=start_detection, font=font_buttons, bg="#43B581", fg="white", relief="flat")
start_button.pack(pady=10)

stop_button = tk.Button(window, text="Stop Detection", command=stop_detection, font=font_buttons, bg="#F04747", fg="white", relief="flat")
stop_button.pack(pady=10)

# Status Label
status_label = Label(window, text="Status: Idle", font=font_status, bg="#2C2F33", fg="white")
status_label.pack(pady=20)

# Run the GUI
window.mainloop()



0: 448x640 8 persons, 1 car, 87.8ms
Speed: 6.0ms preprocess, 87.8ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 7 persons, 1 sports ball, 104.5ms
Speed: 10.0ms preprocess, 104.5ms inference, 12.9ms postprocess per image at shape (1, 3, 448, 640)
